In [1]:
from nltk import ngrams
from nltk import sent_tokenize as tokenize_sent,word_tokenize as tokenize_word
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from string import punctuation
from nltk.probability import FreqDist

def main(ngram):
    x=0
    if(ngram==0):
        print("testing Bigrams language model")
        print("Program to estimate probability of a given sentence")
        while(int(ngram)<1 or int(ngram)>3):
            ngram=input("Please choose [1|2|3] - n gram language model: ") 
            if(ngram.isdigit() == False):
                print("Input must be integer.")
                ngram=0    
    print("===================================================")
    print("1. Compute Probability")
    print("2. Compute Perplexity")
    print("3. Smoothing using Laplace method")
    print("4. exit")
    while(int(x)<1 or int(x)>4):
         x = input("choose: ")
         if(x.isdigit() == False):
             print("Input must be integer.")
             x=0
    return ngram,x
def function_prob(ngram):
    val_perplexity=1
    sentences=input("Enter your text or sentences: ")
    words=[]
    if(ngram==3):   
        new_sent=""
        for se in tokenize_sent(sentences):
            a=se.replace('.','')
            k="start "+"start "+a+" end. "
            new_sent=new_sent+k
        sentences=new_sent
    if(ngram==2):   
        new_sent=""
        for se in tokenize_sent(sentences):
            a=se.replace('.','')
            k="start "+a+" end. "
            new_sent=new_sent+k
    if(ngram==1):
        new_sent=sentences
    for w in tokenize_word(new_sent):
        if w not in punctuation:
            words.append(w)
    fd=FreqDist(words)
    tt=[]
    for se in tokenize_sent(new_sent):
        se=se.replace('.','')
        tt.append(list(map(str.lower,tokenize_word(se))))
    train,padded=padded_everygram_pipeline(ngram,tt)
    model=MLE(ngram)
    model.fit(train,padded)
    test_sent=input("Enter your test sentence: ")
    c=tokenize_sent(test_sent)
    score=[]
    score_2=[]
    sent_temp=[]
    
    prob=1
    if(ngram==1):
        for m in range(len(c)):
            u=c[m].replace('.','')
            test_sent=u.lower()
            temp=[]
            for sent in tokenize_sent(test_sent):
                for w in tokenize_word(sent):
                    if w not in punctuation:
                        temp.append(w)
            for i in range (len(temp)):
                m=model.score(temp[i])
                prob=prob*m
            
                score.append(m)
            if(prob!=0):
                val_perplexity=(prob)**(-1/(len(temp)))
            else:
                val_perplexity=0.0
            score_2.append(val_perplexity)
            sent_temp.append(test_sent)
            print("Probability of  '"+test_sent+"' is: "+str(prob))
            
    
    if(ngram==2):
        for m in range(len(c)):
            prob=1
            u=c[m].replace('.','')
            test_sent="start "+u.lower()+" end. "
            temp=[]
            for sent in tokenize_sent(test_sent):
                for w in tokenize_word(sent):
                    if w not in punctuation:
                        temp.append(w)
            for i in range (len(temp)-1):
                m=model.score(temp[i+1],temp[i].split())
                prob=prob*m
                score.append(m)
            if(prob!=0.0):
                val_perplexity=(prob)**(-(1/(len(temp)-1)))
            else:
                val_perplexity=0.0
            score_2.append(val_perplexity)
            sent_temp.append(test_sent)
            print("Probability of  '"+test_sent+"' is: "+str(prob))
          
    if(ngram==3):
        test_sent_=""
        string=""
        for m in range(len(c)):
            u=c[m].replace('.','')
            k="start "+"start "+u.lower()+" end. "
            test_sent_=test_sent_+k
            temp=[]
            for sent in tokenize_sent(test_sent_):
                for w in tokenize_word(sent):
                    if w not in punctuation:
                        temp.append(w)
            for i in range (2,len(temp)):
                string=string+temp[i-2]+' '+temp[i-1]
                m=model.score(temp[i],string.split())
                score.append(m)
                prob=prob*m
                string=""
            print("Probability of  '"+test_sent_+"' is: "+str(prob))
            sent_temp.append(test_sent_)
            if(prob!=0.0):
                val_perplexity=prob**(-(1/(len(temp)-2)))
            else:
                val_perplexity=0.0
            score_2.append(val_perplexity)
            test_sent_=""
            
    return fd,sent_temp,score,score_2,sentences,temp,ngram


def Smoothing_2(sent,score,score_2,sentences,temp,fd):  
    temp=[]
    k=[]
    s=[]
    temp_prob=[]
    for i, j in fd.most_common():
        k.append(i)
        s.append(j)
    temp3=[]
    a=0
    for y in range (len(sent)):
        probs=1
        for w in tokenize_word(sent[y]):
            if w not in punctuation:
                temp.append(w)
        b=a+len(temp)-1
        for i in range(a,b):
            temp_prob.append(score[i])
        for u in range(len(temp)-1):
            for t in range(len(k)):
                if(k[t]==temp[u]):
                    c=temp_prob[u]*s[t]
                    score_1=(c+1)/(len(k)+s[t])
                    probs*=score_1
                    temp3.append(score_1)
                    break
        a=len(temp)-1
        temp_prob=[]
        temp=[]
        print("Probability of  '"+sent[y]+"' after smoothing is: "+str(probs))                                 
          
def Smoothing_1(sent,score,sentences,fd):
    probs=1
    k=[]
    s=[]
    for i,j in fd.most_common():
        k.append(i)
        s.append(j)
    words=[]
    for w in tokenize_word(sentences):
        if w not in punctuation:
                words.append(w)
    temp=[]
    temp_prob=[]
    a=0
    temp3=[]
    
    for y in range(len(sent)):
        for w in tokenize_word(sent[y]):
                if w not in punctuation:
                    temp.append(w)
        b=a+len(temp)
        for i in range(a,b):
            temp_prob.append(score[i])
        for u in range(len(temp)):
            for t in range(len(k)):
                if(k[t]==temp[u]):    
                    score_1=(s[t]+1)/(len(words)+len(s))
                    temp3.append(score_1)
                    probs*=score_1
                    break
        a=len(temp)
        temp_prob=[]
        temp=[]
        print("Probability of  '"+sent[y]+"' after smoothing is: "+str(probs))                                 
        probs=1
   
    
def Smoothing_3(sent,score,sentences,fd):
    words=[]
    for w in tokenize_word(sentences):
        if w not in punctuation:
               words.append(w)
    s=[]
    for i,j in fd.most_common():
        s.append(j)
    a=0
    y=0
    a=0
    score_=[]
    for p in range(len(sent)):
        k=[]
        probs=1
        count=[]
        val=[]
        for u in tokenize_word(sent[p]):
            if u not in punctuation:
                k.append(u)
        b=a+len(k)-2
        for g in range(a,b):
            val.append(score[g])
        for a in range(len(k)-2):
            m=k[a]
            c=k[a+1]
            for t in range(len(words)-1):
                if(words[t]==m and words[t+1]==c):
                    y+=1
            count.append(y)
            y=0        
        for a in range(len(k)-2):
            d=val[a]*count[a]
            f=(d+1)/(len(s)+count[a])
            probs*=f
            score_.append(f)
        a=len(k)-2
        print("Probability of  '"+sent[y]+"' after smoothing is: "+str(probs))                                 
    
    
if __name__=="__main__":
    ngram=0
    key_word=[]
    score=[]
    m=0
    ngram,k=main(ngram)
    while(int(k)!=4):
        if(int(k)==1):
          fd,sent,score,score_2,sentences,temp,ngram=function_prob(int(ngram))
          m=1
        if(int(k)==2):
            if(m==0):
                print("Please,choose the first choice first")
                print()
            else:
                for k in range(len(score_2)):
                    if(score_2[k]==0.0):
                        print("Perplexity of sentences  '"+sent[k]+"' is does not exist ")
                    else:
                        print("Perplexity of sentences  '"+sent[k]+"' is : "+str(score_2[k]))
        if(int(k)==3):
            if(m==0):
                print("Please,choose the first choice first")
                print()
            else:
                if(ngram==2):
                    Smoothing_2(sent,score,score_2,sentences,temp,fd)
                if(ngram==1):
                    Smoothing_1(sent,score,sentences,fd)
                if(ngram==3):
                    Smoothing_3(sent,score,sentences,fd)
             
        ngram,k=main(ngram)
    

testing Bigrams language model
Program to estimate probability of a given sentence
Please choose [1|2|3] - n gram language model: 2
1. Compute Probability
2. Compute Perplexity
3. Smoothing using Laplace method
4. exit
choose: 1
Enter your text or sentences: i am Wilsen. i like eat bread. i do not like eat rice.
Enter your test sentence: i like eat rice. i am not Wilsen.
Probability of  'start i like eat rice end. ' is: 0.16666666666666666
Probability of  'start i am not wilsen end. ' is: 0.0
1. Compute Probability
2. Compute Perplexity
3. Smoothing using Laplace method
4. exit
choose: 2
Perplexity of sentences  'start i like eat rice end. ' is : 1.4309690811052556
Perplexity of sentences  'start i am not wilsen end. ' is does not exist 
1. Compute Probability
2. Compute Perplexity
3. Smoothing using Laplace method
4. exit
choose: 3
Probability of  'start i like eat rice end. ' after smoothing is: 0.00024151672503320854
Probability of  'start i am not wilsen end. ' after smoothing is: 